In [0]:
%fs ls /FileStore/tables/hyperAWS_accessKeys.csv

In [0]:
aws_keys_df = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/FileStore/tables/hyperAWS_accessKeys.csv')

aws_keys_df.columns

In [0]:
ACCESS_KEY = aws_keys_df.select('Access key ID').take(1)[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').take(1)[0]['Secret access key']

In [0]:
import  urllib

ENCODED_SECRET_KEY = urllib.parse.quote(string = SECRET_KEY, safe = "")

In [0]:
AWS_S3_BUCKET = 'hypernaag'
MOUNT_NAME = '/mnt/mount_s3'

SOURCE_URL = "s3a://%s:%s@%s" %(ACCESS_KEY,ENCODED_SECRET_KEY, AWS_S3_BUCKET)

In [0]:
dbutils.fs.mount(SOURCE_URL,MOUNT_NAME)

In [0]:
%fs ls '/mnt/mount_s3'

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, BooleanType, DataType, DecimalType, DateType

from pyspark.sql.functions import col

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spotify Data Analysis").getOrCreate()

In [0]:
 spotify_schema = StructType([
    StructField("spotify_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("artists", StringType(), True),
    StructField("daily_rank", IntegerType(), True),
    StructField("daily_movement", IntegerType(), True),
    StructField("weekly_movement", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("snapshot_date", DateType(), True),
    StructField("popularity", IntegerType(), True),
    StructField("is_explicit", BooleanType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("album_name", StringType(), True),
    StructField("album_release_date", DateType(), True),
    StructField("danceability", DecimalType(17, 7), True),
    StructField("energy", DecimalType(17, 7), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", DecimalType(17, 7), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", DecimalType(17, 7), True),
    StructField("acousticness", DecimalType(17, 7), True),
    StructField("instrumentalness", DecimalType(17, 7), True),
    StructField("liveness", DecimalType(17, 7), True),
    StructField("valence", DecimalType(17, 7), True),
    StructField("tempo", DecimalType(17, 7), True),
    StructField("time_signature", IntegerType(), True)
])

In [0]:
spotify_df = spark.read.schema(spotify_schema).format('csv').option("header","true").option("inferSchema","true").load('/mnt/mount_s3/universal_top_spotify.csv')


In [0]:
# total unique songs in the dataset
unique_df = spotify_df.distinct().count()

print(unique_df)

719602


In [0]:
spotify_df.head(2)

Out[9]: [Row(spotify_id='2qSkIjg1o9h3YT9RAgYN75', name='Espresso', artists='Sabrina Carpenter', daily_rank=1, daily_movement=0, weekly_movement=1, country=None, snapshot_date=datetime.date(2024, 5, 5), popularity=97, is_explicit=True, duration_ms=175459, album_name='Espresso', album_release_date=datetime.date(2024, 4, 12), danceability=Decimal('0.7010000'), energy=Decimal('0.7600000'), key=0, loudness=Decimal('-5.4780000'), mode=1, speechiness=Decimal('0.0285000'), acousticness=Decimal('0.1070000'), instrumentalness=Decimal('0.0000654'), liveness=Decimal('0.1850000'), valence=Decimal('0.6900000'), tempo=Decimal('103.9690000'), time_signature=4),
 Row(spotify_id='2GxrNKugF82CnoRFbQfzPf', name='i like the way you kiss me', artists='Artemas', daily_rank=2, daily_movement=0, weekly_movement=1, country=None, snapshot_date=datetime.date(2024, 5, 5), popularity=100, is_explicit=False, duration_ms=142514, album_name='i like the way you kiss me', album_release_date=datetime.date(2024, 3, 19), d

In [0]:
spotify_df.createOrReplaceTempView("spotify_data")


In [0]:
top_song = spark.sql("""
        select 
        country,
        spotify_id,
        name,
        count(*)
        from spotify_data
        where daily_rank <= 10 and 
        snapshot_date >= '2024-01-01'
        group by 1,2,3
        order by 4 desc
        limit 10
""")

top_song.show()

+-------+--------------------+------------+--------+
|country|          spotify_id|        name|count(1)|
+-------+--------------------+------------+--------+
|     CR|7bywjHOc0wSjGGbj0...|        LUNA|     124|
|     GT|0R6NfOiLzLj4O5VbY...|   La Diabla|     124|
|     PK|0TL0LFcwIBF5eX7ar...|        Husn|     124|
|     PA|7iQXYTyuG13aoeHxG...| PERRO NEGRO|     124|
|     PK|5rpCUsEfBLIumvrxr...|      Wishes|     124|
|     BO|7bywjHOc0wSjGGbj0...|        LUNA|     124|
|     FR|1eldTykrnkEBLX41b...| Petit génie|     124|
|     UA|1YeQGpxrQvp6Yg3K6...|Покохай мене|     124|
|     ID|7F4tV8SiUy6itZTdA...|penjaga hati|     124|
|     HK|1xRMtDxiU8xxAlaDs...|          濤|     124|
+-------+--------------------+------------+--------+



In [0]:
print(spotify_df.count())

spotify_df.dropDuplicates()  

print(spotify_df.count())

719603
719603


In [0]:
print(spotify_df.count())

spotify_df.dropna()  

print(spotify_df.count())

719603
719603


In [0]:
from pyspark.sql.functions import col, month, desc, rank, year
from pyspark.sql.window import Window

spotify_df = spotify_df.withColumn("month", month("snapshot_date"))

spotify_df = spotify_df.withColumn("year", year("snapshot_date"))

artist_counts = spotify_df.groupBy("year","month", "artists").count()

window_spec = Window.partitionBy("year","month").orderBy(desc("count"))
ranked_artists = artist_counts.withColumn("rank", rank().over(window_spec))

top_10_artists_monthwise = ranked_artists.filter((col("rank") <= 10) & (col("year") >= 2024)).orderBy("year","month", "rank")

top_10_artists_monthwise = top_10_artists_monthwise.na.drop()

top_10_artists_monthwise.show()

+----+-----+--------------------+-----+----+
|year|month|             artists|count|rank|
+----+-----+--------------------+-----+----+
|2024|    1|          Tate McRae| 1541|   1|
|2024|    1|        Taylor Swift| 1175|   2|
|2024|    1|            Doja Cat| 1151|   3|
|2024|    1|         Jack Harlow|  966|   4|
|2024|    1|                Xavi|  965|   5|
|2024|    1|The Weeknd, JENNI...|  954|   6|
|2024|    1|            Dua Lipa|  952|   7|
|2024|    1|         Kenya Grace|  951|   8|
|2024|    1|The Weeknd, Playb...|  938|   9|
|2024|    1|                Feid|  919|  10|
|2024|    2|Kanye West, Ty Do...| 1376|   1|
|2024|    2|          Tate McRae| 1307|   2|
|2024|    2|        Taylor Swift| 1240|   3|
|2024|    2|            Dua Lipa| 1107|   4|
|2024|    2|         Myke Towers|  988|   5|
|2024|    2|        Benson Boone|  968|   6|
|2024|    2|                Xavi|  957|   7|
|2024|    2|The Weeknd, JENNI...|  885|   8|
|2024|    2|             KAROL G|  875|   9|
|2024|    

In [0]:
s3_output_path = "/mnt/mount_s3/output.csv"

top_10_artists_monthwise.write.csv(s3_output_path, mode="overwrite", header=True)

print("CSV file successfully written to S3 location:", s3_output_path)

CSV file successfully written to S3 location: /mnt/mount_s3/output.csv


In [0]:
from pyspark.sql.functions import row_number

spotify_df = spotify_df.withColumn("month", month("snapshot_date"))

spotify_df = spotify_df.withColumn("year", year("snapshot_date"))

artist_counts = spotify_df.groupBy("year","month", "artists", "country").count()

window_spec = Window.partitionBy("year","month", "country").orderBy(desc("count"))
ranked_artists = artist_counts.withColumn("rank", row_number().over(window_spec))

top_10_artists_countrywise = ranked_artists.filter((col("rank") <= 10) & (col("year") >= 2024) & (col("month")== 4)).orderBy("year","month","country","rank")

top_10_artists_countrywise = top_10_artists_countrywise.na.drop()

top_10_artists_countrywise.show()

+----+-----+--------------------+-------+-----+----+
|year|month|             artists|country|count|rank|
+----+-----+--------------------+-------+-----+----+
|2024|    4|        Taylor Swift|     AE|   83|   1|
|2024|    4|       Ariana Grande|     AE|   62|   2|
|2024|    4|                 SZA|     AE|   45|   3|
|2024|    4|                   V|     AE|   35|   4|
|2024|    4|   Sabrina Carpenter|     AE|   32|   5|
|2024|    4|             Artemas|     AE|   26|   6|
|2024|    4|¥$, Kanye West, T...|     AE|   25|   7|
|2024|    4|                Tyla|     AE|   25|   8|
|2024|    4|         Teddy Swims|     AE|   25|   9|
|2024|    4|                 Djo|     AE|   25|  10|
|2024|    4|                TINI|     AR|   90|   1|
|2024|    4|Tiago PZK, Ke Per...|     AR|   30|   2|
|2024|    4|Salastkbron, Diel...|     AR|   30|   3|
|2024|    4|Emanero, Karina, ...|     AR|   30|   4|
|2024|    4|ORO600, Pablo Chi...|     AR|   30|   5|
|2024|    4|Omar Varela, El n...|     AR|   30

In [0]:
s3_output_path = "/mnt/mount_s3/top_artist_countrywise.csv"

# Write the DataFrame to the PDF file in the specified S3 location
top_10_artists_countrywise.write.csv(s3_output_path, mode="overwrite", header=True)

print("PDF file successfully written to S3 location:", s3_output_path)

PDF file successfully written to S3 location: /mnt/mount_s3/top_artist_countrywise.csv


In [0]:
top_dance_song = spark.sql("""
    select distinct
    'April 2024' as dt,
    name,
    artists,
    danceability,
    energy
    from spotify_data
    where snapshot_date >= '2024-04-01' and snapshot_date <= '2024-04-30' 
    and energy >= 0.75
    order by 4 desc, 5 desc
    limit 10
""")
top_dance_song.show()

+----------+--------------------+--------------------+------------+---------+
|        dt|                name|             artists|danceability|   energy|
+----------+--------------------+--------------------+------------+---------+
|April 2024|            Cariceps|Dani Mocanu, Alex...|   0.9630000|0.8090000|
|April 2024|   Țigancă Balcanică|Tzanca Uraganu, M...|   0.9620000|0.7940000|
|April 2024|             Sukarie|Trannos, Thodoris...|   0.9610000|0.9470000|
|April 2024|قمبلة الجيل ( اخت...|Hassan Shakosh, ح...|   0.9510000|0.8780000|
|April 2024|Edit do Chico Ofi...|MC Saci, Mc Pretc...|   0.9510000|0.8240000|
|April 2024|Petit Fou Fou (fe...|         Rhove, ANNA|   0.9480000|0.7560000|
|April 2024|Hoe (feat. Sfera ...|Tedua, Sfera Ebbasta|   0.9360000|0.7570000|
|April 2024|      Temps en temps|      Zola, Koba LaD|   0.9340000|0.8070000|
|April 2024|مهرجان يازميكس ال...|Fares Sokar - فار...|   0.9260000|0.9640000|
|April 2024|            24/7 6.5|YSY A, Jere Klein...|   0.92500

In [0]:
s3_output_path = "/mnt/mount_s3/top_international_dance.csv"

# Write the DataFrame to the CSV file in the specified S3 location
top_dance_song.write.csv(s3_output_path, mode="overwrite", header=True)

print("PDF file successfully written to S3 location:", s3_output_path)

PDF file successfully written to S3 location: /mnt/mount_s3/top_international_dance.csv
